In [1]:
import io
import os
import requests
import pandas as pd
from google.cloud import storage

# code_kokoro\terra_form_mystic-shelter-339813-86fc09673374.json

## Set up your enviroment credentials

In [8]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\orena\OneDrive\01_ORENAIKE\02_CAREER_AND_DEVELOPMENTS\SkillsIT\Data_Engineering2024\code_kokoro\terra_form_mystic-shelter-339813-86fc09673374.json"

storage_client = storage.Client()

## create a bucket name

In [2]:
# create a new bucket
# import packages
from google.cloud import storage
import os

# set key credentials file path
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\orena\OneDrive\01_ORENAIKE\02_CAREER_AND_DEVELOPMENTS\SkillsIT\Data_Engineering2024\code_kokoro\terra_form_mystic-shelter-339813-86fc09673374.json"

# define function that creates the bucket
def create_bucket(bucket_name, storage_class='STANDARD', location='us-central1'): 
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    bucket.storage_class = storage_class
   
    bucket = storage_client.create_bucket(bucket, location=location) 
    # for dual-location buckets add data_locations=[region_1, region_2]
    
    return f'Bucket {bucket.name} successfully created.'

## Invoke Function
print(create_bucket('nyc-green-taxi-data_v1', 'STANDARD', 'us-central1'))

Bucket nyc-green-taxi-data_v1 successfully created.


## Upload a file

In [ ]:
# import packages
from google.cloud import storage
import os

# # set key credentials file path
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'D:/VSCode/GitRepos/PythonHacks/GCP_Storage_Bucket_Handling_With_Python/cloudquicklabs-93d1e8c6ac6a.json'

# define function that uploads a file from the bucket
def upload_cs_file(bucket_name, source_file_name, destination_file_name): 
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)

    blob = bucket.blob(destination_file_name)
    blob.upload_from_filename(source_file_name)

    return True

upload_cs_file('test_demo_storage_bucket', 'D:/VSCode/GitRepos/PythonHacks/GCP_Storage_Bucket_Handling_With_Python/cloudquicklabs-93d1e8c6ac6a.json', 'json/test.json')

## Upload Parquet Files for green taxi

In [3]:
import os
import requests
from google.cloud import storage

# No need to switch out bucket name, assuming it's already set correctly
BUCKET = os.environ.get("GCP_GCS_BUCKET", "nyc-green-taxi-data_v1")

def upload_to_gcs(bucket, object_name, local_file):
    """
    Uploads a file to Google Cloud Storage.
    """
    client = storage.Client()
    bucket = client.bucket(bucket)
    blob = bucket.blob(object_name)
    blob.upload_from_filename(local_file)

def download_file(url, local_file_path):
    """
    Downloads a file from a given URL.
    """
    response = requests.get(url)
    with open(local_file_path, 'wb') as file:
        file.write(response.content)
    print(f"Downloaded {local_file_path}")

def web_to_gcs(year):
    base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_"
    for i in range(1, 13):
        month = f"{i:02d}"  # Format month as two digits
        file_name = f"{year}-{month}.parquet"  # File name for the local file
        url = f"{base_url}{file_name}"  # Construct the URL

        # Download the .parquet file directly
        download_file(url, file_name)

        # Upload it to GCS
        upload_to_gcs(BUCKET, f"green/{file_name}", file_name)
        print(f"Uploaded to GCS: green/{file_name}")

# Example usage for 2022 data
web_to_gcs('2022')


Downloaded 2022-01.parquet
Uploaded to GCS: green/2022-01.parquet
Downloaded 2022-02.parquet
Uploaded to GCS: green/2022-02.parquet
Downloaded 2022-03.parquet
Uploaded to GCS: green/2022-03.parquet
Downloaded 2022-04.parquet
Uploaded to GCS: green/2022-04.parquet
Downloaded 2022-05.parquet
Uploaded to GCS: green/2022-05.parquet
Downloaded 2022-06.parquet
Uploaded to GCS: green/2022-06.parquet
Downloaded 2022-07.parquet
Uploaded to GCS: green/2022-07.parquet
Downloaded 2022-08.parquet
Uploaded to GCS: green/2022-08.parquet
Downloaded 2022-09.parquet
Uploaded to GCS: green/2022-09.parquet
Downloaded 2022-10.parquet
Uploaded to GCS: green/2022-10.parquet
Downloaded 2022-11.parquet
Uploaded to GCS: green/2022-11.parquet
Downloaded 2022-12.parquet
Uploaded to GCS: green/2022-12.parquet


## SQL for Creating External table in Big Query 

In [ ]:
-- Creating external table referring to gcs path
CREATE OR REPLACE EXTERNAL TABLE `mystic-shelter-339813.ny_taxi.nyc-green-taxi-data_2022_v1`
OPTIONS (
  format = 'parquet',
  uris = ['gs://nyc-green-taxi-data_v1/green/2022-*.parquet']
);


SELECT * FROM `mystic-shelter-339813.ny_taxi.nyc-green-taxi-data_2022_v1` LIMIT 10;

-- Q1
-- Question 1: What is count of records for the 2022 Green Taxi Data?? 
SELECT COUNT(*) FROM `mystic-shelter-339813.ny_taxi.nyc-green-taxi-data_2022_v1`;
-- 840402

-- Q2
-- Write a query to count the distinct number of PULocationIDs for the entire dataset on both the tables.
SELECT COUNT(DISTINCT PULocationID) 
FROM (
  SELECT PULocationID FROM `mystic-shelter-339813.ny_taxi.nyc-green-taxi-data_2022_v1`
  UNION ALL
  SELECT PULocationID FROM `mystic-shelter-339813.ny_taxi.nyc-green-taxi-data_2022_v1`
) AS combined_tables;


-- Q3
-- How many records have a fare_amount of 0?
SELECT COUNT(*) AS zero_fare_records
FROM `mystic-shelter-339813.ny_taxi.nyc-green-taxi-data_2022_v1`
WHERE fare_amount = 0;
-- 1622



-- Q4


-- Q5
-- Write a query to retrieve the distinct PULocationID between lpep_pickup_datetime 06/01/2022 and 06/30/2022 (inclusive)

-- Q6
-- Where is the data stored in the External Table you created?


-- Q7
-- It is best practice in Big Query to always cluster your data:
-- False


-- Write a SELECT count(*) query FROM the materialized table you created. How many bytes does it estimate will be read? Why?
